In [7]:
import pandas as pd
import numpy as np
import requests
''' 
Шаг 1: Загрузка данных: Мы используем библиотеку requests, чтобы скачать файл CSV с данными о рыбах, которые смотрим, но не продаём.
Потом сохраняем его локально в файл с кодировкой UTF-8 (чтобы избежать проблем с символами).
'''
url = 'https://raw.githubusercontent.com/YBI-Foundation/Dataset/refs/heads/main/Fish.csv'
r = requests.get(url)
with open('Fish.csv', 'w', encoding='utf-8') as f:
    f.write(r.text)

# Чтение данных
df = pd.read_csv('Fish.csv')

''' 
Шаг 2: Разделение данных: Мы делим датасет на две части: первая половина строк идёт в одну часть, вторая половина – в другую. 
Затем объединяем их обратно с помощью pd.concat.
'''
part1 = df[:len(df)//2]
part2 = df[len(df)//2:]
df_combined = pd.concat([part1, part2], ignore_index=True)

''' Шаг 3: Мы проверяем, есть ли в данных дубликаты, используя duplicated().sum(). 
Это показывает, сколько строк повторяются. Если дубликаты есть, мы их удаляем с помощью drop_duplicates(). 
'''
print(f"Количество дубликатов до удаления: {df_combined.duplicated().sum()}")
df_combined = df_combined.drop_duplicates()
print(f"Количество дубликатов после удаления: {df_combined.duplicated().sum()}")

''' 
Шаг 4: Если в данных нет пропусков, мы специально добавляем несколько случайных пропусков (в столбцы Weight и Height). 
'''
print(f"Количество пропусков до добавления: {df_combined.isnull().sum().sum()}")

if df_combined.isnull().sum().sum() == 0:
    df_combined.loc[df_combined.sample(frac=0.05).index, 'Weight'] = np.nan  # Добавляем пропуски в столбец Weight
    df_combined.loc[df_combined.sample(frac=0.03).index, 'Height'] = np.nan  # Добавляем пропуски в столбец Height

print(f"Количество пропусков после добавления: {df_combined.isnull().sum().sum()}")

# Шаг 5: Заполнение пропусков (я сделал оба вариант фана ради)
'''
Пропуски в числовых столбцах (например, Weight) мы заполняем медианой — это среднее значение, которое более устойчиво к выбросам, 
чем обычная средняя арифметическая. А в категориальных столбцах - заполняем пропуски самым частым значением, то есть модой.
'''
# Для числовых переменных: медиана
for col in df_combined.select_dtypes(include=[np.number]).columns:
    df_combined[col] = df_combined[col].fillna(df_combined[col].median())

# Для категориальных переменных: мода
for col in df_combined.select_dtypes(include=['object']).columns:
    df_combined[col] = df_combined[col].fillna(df_combined[col].mode()[0])
'''
Шаг 6: Мы используем межквартильный размах (IQR), чтобы найти выбросы (очень большие или очень маленькие значения) и удаляем их. 
'''
for col in df_combined.select_dtypes(include=[np.number]).columns:
    Q1 = df_combined[col].quantile(0.25)
    Q3 = df_combined[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_combined = df_combined[(df_combined[col] >= lower_bound) & (df_combined[col] <= upper_bound)]
'''
Шаг 7: Числовые данные нормализуем с помощью метода мин-макс нормализации. Это приводит все значения к диапазону от 0 до 1. 
'''
for col in df_combined.select_dtypes(include=[np.number]).columns:
    df_combined[col] = (df_combined[col] - df_combined[col].min()) / (df_combined[col].max() - df_combined[col].min())

# Результат
df_combined.head()


Количество дубликатов до удаления: 0
Количество дубликатов после удаления: 0
Количество пропусков до добавления: 0
Количество пропусков после добавления: 13


,Category,Species,Weight,Height,Width,Length1,Length2,Length3
0,0.0,Bream,0.220000,0.568334,0.418978,0.384804,0.392610,0.457883
1,0.0,Bream,0.263636,0.624055,0.459235,0.404412,0.413395,0.483801
2,0.0,Bream,0.309091,0.618123,0.514279,0.401961,0.418014,0.481641
3,0.0,Bream,0.330000,0.638566,0.480365,0.460784,0.475751,0.533477
4,0.0,Bream,0.390909,0.621966,0.576004,0.465686,0.475751,0.544276
